In [1]:
from sklearn.ensemble import RandomForestClassifier
from preprocessing import *
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn
import mlflow.tracking
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from skopt import BayesSearchCV

/Users/alejandrovaca/anaconda3/envs/sentinel_env/lib/python3.7/imp.py:342: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  return _load(spec)


In [2]:
NAME = 'catboost1'

In [3]:
mlflow.start_run(run_name=NAME)

<ActiveRun: >

In [4]:
import sklearn
#sorted(sklearn.metrics.SCORERS.keys())

In [5]:
params = {'depth': (1, 10),
         'iterations': (100, 1000),
         'l2_leaf_reg': (1, 20),
         'learning_rate': (1e-6, 5e-1, 'log-uniform')}

In [6]:
params_rf = {'n_estimators': (100, 1500),
            'max_depth': (2, 10),
            'class_weight': ['balanced'],
            'n_jobs': [-1],
            'max_features': ['auto', 'log2'],
            }

In [24]:
data = pd.read_csv('Modelar_UH2020.txt', sep='|')
#catboost = CatBoostClassifier(iterations=100, learning_rate=0.05, depth=7, cat_features=)
#cf = CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 50, l2_leaf_reg= 3, learning_rate= 0.1)
cf = RandomForestClassifier(n_estimators=50)

In [25]:
#data.loc[:, data.dtypes==object]


In [30]:
X, y = preprocess_data('Modelar_UH2020.txt', process_cat=True, sample_trials=10000)

El diccionario de categorías es {'INDUSTRIAL': 0, 'OTHER': 1, 'AGRICULTURE': 2, 'OFFICE': 3, 'PUBLIC': 4, 'RESIDENTIAL': 5, 'RETAIL': 6}


/Users/alejandrovaca/anaconda3/envs/sentinel_env/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [31]:
X.shape, y.shape

((10000, 87), (10000,))

In [34]:
y = y.astype('int')


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [36]:
cf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
cf_prob = cf.predict_proba(X_test)

In [14]:
cf_pred = cf.predict(X_test)

In [15]:
cf.score(X_test, y_test)

0.8936

In [16]:
setlabs = [l for l in set(y_train)]

In [17]:
setlabs

[0, 1, 2, 3, 4, 5, 6]

In [18]:
tag2idx = {i: l for l, i
           in enumerate(setlabs)}

In [19]:
tag2idx

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}

In [20]:
#cf_pred = [p[0] for p in cf_pred]
#cf_pred

In [21]:
#preds_num = [tag2idx[v] for v in cf_pred]
#true_num = [tag2idx[v] for v in y_test]

In [22]:
set(cf_pred)

TypeError: unhashable type: 'numpy.ndarray'

In [23]:
cm = confusion_matrix(y_test, cf_pred)

ValueError: Classification metrics can't handle a mix of unknown and multiclass targets

In [ ]:
cm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14, normalize=True):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    if normalize:
        confusion_matrix = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    fmt = '.2f' if normalize else 'd'
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt=fmt)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return fig

In [ ]:
grafico_conf_matrix = print_confusion_matrix(cm, class_names = setlabs,)

In [ ]:
grafico_conf_matrix.savefig(NAME)

In [ ]:
print_confusion_matrix(cm, class_names = setlabs, normalize=False)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
mlflow.log_metrics(metrics={'f1': f1_score(y_test, cf_pred, average='macro'),
                           'precision': precision_score(y_test, cf_pred, average='macro'),
                           'recall': recall_score(y_test, cf_pred, average='macro'),
                           'accuracy': accuracy_score(y_test, cf_pred)})

In [ ]:
mlflow.log_artifact(f'./{NAME}.png')

In [ ]:
#mlflow.end_run()